In [2]:
"""
This script demonstrates the conversion from RGB to HSV color space and manually calculates the Hue (H) channel.

### Key Concepts:

1. **Color Spaces:**
   - **RGB (Red, Green, Blue)**: The standard color model used to represent colors by combining different intensities of red, green, and blue.
   - **HSV (Hue, Saturation, Value)**: A cylindrical color model used in computer graphics and image processing. It represents:
     - **Hue (H)**: The type of color (e.g., red, blue, yellow) represented as an angle on the color wheel.
     - **Saturation (S)**: The intensity or purity of the color, where 0 is grayscale and 1 is the purest form of the color.
     - **Value (V)**: The brightness or intensity of the color, where 0 is completely dark (black) and 1 is fully bright.

2. **Conversion from RGB to HSV:**
   - OpenCV's `cvtColor` function is used to convert RGB values into the HSV color space. The Hue (H) in OpenCV is scaled to a range of [0, 179], while it typically ranges from [0, 360] in other systems.

3. **Manual Calculation of Hue (H):**
   - **Hue Calculation**: The Hue (H) in HSV is determined by finding the dominant color component (Red, Green, or Blue) and calculating an angle on the color wheel.
   - The formula for Hue is as follows:
     - For **C_max = R** (when the Red component is the maximum):
       - 'H = 60 * ((G - B) / Δ) % 6'
     - For **C_max = G** (when the Green component is the maximum):
       - 'H = 60 * ((B - R) / Δ + 2)'
     - For **C_max = B** (when the Blue component is the maximum):
       - 'H = 60 * ((R - G) / Δ + 4)'
     
     Where:
     - 'C_max' is the maximum of the R, G, and B components.
     - 'C_min' is the minimum of the R, G, and B components.
     - 'Δ (delta)' is the difference: 'Δ = C_max - C_min'.
     
   - **When Δ = 0** (i.e., the color is grayscale), Hue is undefined, and OpenCV sets it to 0.
   
   - The calculated Hue is then scaled to fit the OpenCV range of [0, 179] by dividing the result by 2:
     - 'H_scaled = (H / 2) % 180'

4. **RGB Image Example:**
   - The image is a 3x3 RGB matrix containing:
     - Primary colors: Red, Green, Blue
     - Secondary colors: Yellow, Cyan, Magenta
     - Shades of gray: Different intensities of gray.

### Program Workflow:
1. **Define a 3x3 RGB image**: A small sample image with various colors and shades of gray.
2. **Convert RGB to HSV**: Use OpenCV to convert the image to HSV and extract the Hue (H) channel.
3. **Manual Hue Calculation**: Manually calculate the Hue (H) using the RGB components of the image.
4. **Compare Results**: Display the Hue (H) values from OpenCV and manually calculated Hue to compare the results.

### Example Use Cases:
- **Color-Based Segmentation**: Hue (H) can be used for segmenting images based on color (e.g., isolating red, blue, or green regions).
- **Image Processing**: Understanding how the Hue channel is calculated and scaled helps in manipulating colors in images, such as for color correction or enhancement.

"""

import numpy as np
import cv2

# Define the 3x3 RGB image
image = np.array([
    [[255, 0, 0], [0, 255, 0], [0, 0, 255]],       # Row 1: Red, Green, Blue
    [[255, 255, 0], [0, 255, 255], [255, 0, 255]], # Row 2: Yellow, Cyan, Magenta
    [[128, 128, 128], [64, 64, 64], [32, 32, 32]]  # Row 3: Shades of gray
], dtype=np.uint8)

# Convert to HSV using OpenCV
hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

# Extract Hue (H) channel from OpenCV
H_channel_opencv = hsv_image[:, :, 0]

# Manual calculation of Hue (H)
R, G, B = image[:, :, 0], image[:, :, 1], image[:, :, 2]
C_max = np.max(image, axis=2)
C_min = np.min(image, axis=2)
delta = C_max - C_min

H_channel_manual = np.zeros_like(C_max, dtype=np.float32)

# Avoid division by zero
non_zero_delta = delta > 0

# When C_max == R
mask = (C_max == R) & non_zero_delta
H_channel_manual[mask] = 60 * ((G[mask] - B[mask]) / delta[mask] % 6)

# When C_max == G
mask = (C_max == G) & non_zero_delta
H_channel_manual[mask] = 60 * ((B[mask] - R[mask]) / delta[mask] + 2)

# When C_max == B
mask = (C_max == B) & non_zero_delta
H_channel_manual[mask] = 60 * ((R[mask] - G[mask]) / delta[mask] + 4)

# Scale H to OpenCV range [0, 179]
H_channel_manual = (H_channel_manual / 2) % 180
H_channel_manual = H_channel_manual.astype(np.uint8)

# Compare results
print("Hue (H) from OpenCV:")
print(H_channel_opencv)

print("\nManually Calculated Hue (H):")
print(H_channel_manual)


Hue (H) from OpenCV:
[[  0  60 120]
 [ 30  90 150]
 [  0   0   0]]

Manually Calculated Hue (H):
[[  0  60 120]
 [ 60 120 150]
 [  0   0   0]]
